# Using Word2Vec for Topic Modeling

The idea of using an $N$ dimensional feature vector for unsupervised learning a powerful concept.  If we can take a body of words and consider them as a vector, we can begin to interpret words and sentences as points in $N$ dimensional space, and use traditional methods, including the K Means algorithm, to assign labels to unstructured data.

In fact, the most complex algorithm in this notebook is *not* the K Means algorithm, but rather the preparation of the words to be used by the algorithm, which is the Word2Vec algorithm.  Word2Vec is a member of the class of algorithms that carry out a *word embedding*, which simply means that we take a collection of words and return a feature vector.  The feature vector is always of the same dimension, even though the collection of words may not be.   

An amazing property of word2vec is that vector embeddings can often result in interesting geometric interpretations, including additive properties like 

$$\text{w2v}(king) - \text{w2v}(queen) = \text{w2v}(man) - \text{w2v}(woman)$$, 

which suggests that an article, or body of text, can be a superposition of word vectors whose position in feature space relative to other articles can indicate their similarity.  This is a very helpful insight.

Let's take a look at this algorithm in action.  First, let's import some libraries:

In [1]:
// command line operations
import scala.sys.process._

// Spark Classes
import scala.collection.mutable
//import org.apache.spark.mllib.clustering.LDA
//import org.apache.spark.mllib.linalg.{Vector, Vectors}
import org.apache.spark.rdd.RDD
import org.apache.spark.ml.clustering.KMeans
import org.apache.spark.ml.feature.Word2Vec
import org.apache.spark.mllib.linalg.{Vector, Vectors}

// .tar and .gz utilities
import org.apache.commons.compress.compressors.gzip.GzipCompressorInputStream
import org.apache.commons.compress.archivers.tar.TarArchiveInputStream
import org.apache.commons.compress.compressors.gzip.GzipCompressorInputStream
import org.apache.commons.compress.archivers.tar.TarArchiveInputStream
import org.apache.spark.input.PortableDataStream
import scala.util.Try
import java.nio.charset._

// rounding function for time:
def round(x: Long) ={Math.round(x/1e6)/1e3}

round: (x: Long)Double


# Loading the Corpus

We will download the yahoo 20 newsgroups dataset if you haven't already done so. This dataset contains 20,000 news stories that have been manually categorized into directories.  

In [2]:
import sys.process._
val listing = "ls 20_newsgroups"!! ;

if (listing.contains("20_newsgroups") == false ) {
    println("downloading dataset")
    "wget http://qwone.com/~jason/20Newsgroups/20news-19997.tar.gz"!;
    "tar -xvf 20news-19997.tar.gz"!;
    "rm 20news-19997*"!;
}
println("directories in 20 newsgroup dataset")


downloading dataset
--2019-12-10 05:46:25--  http://qwone.com/~jason/20Newsgroups/20news-19997.tar.gz
Resolving qwone.com (qwone.com)... 108.20.201.166
Connecting to qwone.com (qwone.com)|108.20.201.166|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17332201 (17M) [application/x-gzip]
Saving to: ‘20news-19997.tar.gz.1’

     0K .......... .......... .......... .......... ..........  0%  101K 2m47s
    50K .......... .......... .......... .......... ..........  0% 92.7K 2m54s
   100K .......... .......... .......... .......... ..........  0%  779K 2m3s
   150K .......... .......... .......... .......... ..........  1%  246K 1m49s
   200K .......... .......... .......... .......... ..........  1%  112K 1m57s
   250K .......... .......... .......... .......... ..........  1%  186K 1m52s
   300K .......... .......... .......... .......... ..........  2%  163K 1m50s
   350K .......... .......... .......... .......... ..........  2%  242K 1m45s
   400K .......... ..

  5050K .......... .......... .......... .......... .......... 30%  197K 64s
  5100K .......... .......... .......... .......... .......... 30%  173K 64s
  5150K .......... .......... .......... .......... .......... 30%  173K 64s
  5200K .......... .......... .......... .......... .......... 31%  184K 64s
  5250K .......... .......... .......... .......... .......... 31%  151K 63s
  5300K .......... .......... .......... .......... .......... 31%  190K 63s
  5350K .......... .......... .......... .......... .......... 31%  187K 63s
  5400K .......... .......... .......... .......... .......... 32%  191K 63s
  5450K .......... .......... .......... .......... .......... 32%  199K 62s
  5500K .......... .......... .......... .......... .......... 32%  188K 62s
  5550K .......... .......... .......... .......... .......... 33%  196K 62s
  5600K .......... .......... .......... .......... .......... 33%  195K 61s
  5650K .......... .......... .......... .......... .......... 33%  193K 61s

x 20_newsgroups/alt.atheism/51161
x 20_newsgroups/alt.atheism/51162
x 20_newsgroups/alt.atheism/51163
x 20_newsgroups/alt.atheism/51164
x 20_newsgroups/alt.atheism/51165
x 20_newsgroups/alt.atheism/51246
x 20_newsgroups/alt.atheism/51257
x 20_newsgroups/alt.atheism/51258
x 20_newsgroups/alt.atheism/51259
x 20_newsgroups/alt.atheism/51260
x 20_newsgroups/alt.atheism/51261
x 20_newsgroups/alt.atheism/51262
x 20_newsgroups/alt.atheism/51285
x 20_newsgroups/alt.atheism/51264
x 20_newsgroups/alt.atheism/51265
x 20_newsgroups/alt.atheism/51266
x 20_newsgroups/alt.atheism/51267
x 20_newsgroups/alt.atheism/51268
x 20_newsgroups/alt.atheism/51269
x 20_newsgroups/alt.atheism/51270
x 20_newsgroups/alt.atheism/51271
x 20_newsgroups/alt.atheism/51284
x 20_newsgroups/alt.atheism/51286
x 20_newsgroups/alt.atheism/51287
x 20_newsgroups/alt.atheism/51288
x 20_newsgroups/alt.atheism/51289
x 20_newsgroups/alt.atheism/51290
x 20_newsgroups/alt.atheism/51291
x 20_newsgroups/alt.atheism/53270
x 20_newsgroup

x 20_newsgroups/alt.atheism/51225
x 20_newsgroups/alt.atheism/51226
x 20_newsgroups/alt.atheism/51227
x 20_newsgroups/alt.atheism/51228
x 20_newsgroups/alt.atheism/49960
x 20_newsgroups/alt.atheism/51303
x 20_newsgroups/alt.atheism/51304
x 20_newsgroups/alt.atheism/51305
x 20_newsgroups/alt.atheism/51306
x 20_newsgroups/alt.atheism/51307
x 20_newsgroups/alt.atheism/51308
x 20_newsgroups/alt.atheism/51309
x 20_newsgroups/alt.atheism/51310
x 20_newsgroups/alt.atheism/51311
x 20_newsgroups/alt.atheism/51312
x 20_newsgroups/alt.atheism/51313
x 20_newsgroups/alt.atheism/51314
x 20_newsgroups/alt.atheism/51315
x 20_newsgroups/alt.atheism/51316
x 20_newsgroups/alt.atheism/51317
x 20_newsgroups/alt.atheism/51318
x 20_newsgroups/alt.atheism/51319
x 20_newsgroups/alt.atheism/51320
x 20_newsgroups/alt.atheism/53660
x 20_newsgroups/alt.atheism/53661
x 20_newsgroups/alt.atheism/53665
x 20_newsgroups/alt.atheism/53611
x 20_newsgroups/alt.atheism/53612
x 20_newsgroups/alt.atheism/53666
x 20_newsgroup

x 20_newsgroups/alt.atheism/53286
x 20_newsgroups/alt.atheism/53287
x 20_newsgroups/alt.atheism/53224
x 20_newsgroups/alt.atheism/53225
x 20_newsgroups/alt.atheism/53226
x 20_newsgroups/alt.atheism/53227
x 20_newsgroups/alt.atheism/53228
x 20_newsgroups/alt.atheism/53229
x 20_newsgroups/alt.atheism/53230
x 20_newsgroups/alt.atheism/53231
x 20_newsgroups/alt.atheism/53232
x 20_newsgroups/alt.atheism/53233
x 20_newsgroups/alt.atheism/53234
x 20_newsgroups/alt.atheism/53235
x 20_newsgroups/alt.atheism/53236
x 20_newsgroups/alt.atheism/53288
x 20_newsgroups/alt.atheism/53289
x 20_newsgroups/alt.atheism/53290
x 20_newsgroups/alt.atheism/53291
x 20_newsgroups/alt.atheism/53292
x 20_newsgroups/alt.atheism/53293
x 20_newsgroups/alt.atheism/53294
x 20_newsgroups/alt.atheism/53295
x 20_newsgroups/alt.atheism/53296
x 20_newsgroups/alt.atheism/53297
x 20_newsgroups/alt.atheism/53298
x 20_newsgroups/alt.atheism/53299
x 20_newsgroups/alt.atheism/53300
x 20_newsgroups/alt.atheism/53301
x 20_newsgroup

...

x 20_newsgroups/talk.religion.misc/83453
x 20_newsgroups/talk.religion.misc/83454
x 20_newsgroups/talk.religion.misc/83455
x 20_newsgroups/talk.religion.misc/83542
x 20_newsgroups/talk.religion.misc/82760
x 20_newsgroups/talk.religion.misc/82761
x 20_newsgroups/talk.religion.misc/82762
x 20_newsgroups/talk.religion.misc/82763
x 20_newsgroups/talk.religion.misc/82764
x 20_newsgroups/talk.religion.misc/82765
x 20_newsgroups/talk.religion.misc/82766
x 20_newsgroups/talk.religion.misc/82799
x 20_newsgroups/talk.religion.misc/82800
x 20_newsgroups/talk.religion.misc/83456
x 20_newsgroups/talk.religion.misc/82801
x 20_newsgroups/talk.religion.misc/82802
x 20_newsgroups/talk.religion.misc/82803
x 20_newsgroups/talk.religion.misc/82804
x 20_newsgroups/talk.religion.misc/82805
x 20_newsgroups/talk.religion.misc/83457
x 20_newsgroups/talk.religion.misc/83458
x 20_newsgroups/talk.religion.misc/83459
x 20_newsgroups/talk.religion.misc/83460
x 20_newsgroups/talk.religion.misc/83461
x 20_newsgroups/

x 20_newsgroups/talk.religion.misc/83790
x 20_newsgroups/talk.religion.misc/83791
x 20_newsgroups/talk.religion.misc/83881
x 20_newsgroups/talk.religion.misc/83882
x 20_newsgroups/talk.religion.misc/83883
x 20_newsgroups/talk.religion.misc/83884
x 20_newsgroups/talk.religion.misc/83885
x 20_newsgroups/talk.religion.misc/83886
x 20_newsgroups/talk.religion.misc/83946
x 20_newsgroups/talk.religion.misc/83950
x 20_newsgroups/talk.religion.misc/83951
x 20_newsgroups/talk.religion.misc/83952
x 20_newsgroups/talk.religion.misc/83954
x 20_newsgroups/talk.religion.misc/83956
x 20_newsgroups/talk.religion.misc/83957
x 20_newsgroups/talk.religion.misc/83958
x 20_newsgroups/talk.religion.misc/83969
x 20_newsgroups/talk.religion.misc/83970
x 20_newsgroups/talk.religion.misc/83971
x 20_newsgroups/talk.religion.misc/83979
x 20_newsgroups/talk.religion.misc/83980
x 20_newsgroups/talk.religion.misc/83981
x 20_newsgroups/talk.religion.misc/83982
x 20_newsgroups/talk.religion.misc/83983
x 20_newsgroups/

x 20_newsgroups/talk.religion.misc/83682
x 20_newsgroups/talk.religion.misc/83683
x 20_newsgroups/talk.religion.misc/83684
x 20_newsgroups/talk.religion.misc/83685
x 20_newsgroups/talk.religion.misc/83686
x 20_newsgroups/talk.religion.misc/83687
x 20_newsgroups/talk.religion.misc/83688
x 20_newsgroups/talk.religion.misc/83689
x 20_newsgroups/talk.religion.misc/83690
x 20_newsgroups/talk.religion.misc/83691
x 20_newsgroups/talk.religion.misc/83692
x 20_newsgroups/talk.religion.misc/83693
x 20_newsgroups/talk.religion.misc/83694
x 20_newsgroups/talk.religion.misc/83721
x 20_newsgroups/talk.religion.misc/83722
x 20_newsgroups/talk.religion.misc/83723
x 20_newsgroups/talk.religion.misc/83724
x 20_newsgroups/talk.religion.misc/83725
x 20_newsgroups/talk.religion.misc/83726
x 20_newsgroups/talk.religion.misc/83727
x 20_newsgroups/talk.religion.misc/83728
x 20_newsgroups/talk.religion.misc/83729
x 20_newsgroups/talk.religion.misc/83730
x 20_newsgroups/talk.religion.misc/83731
x 20_newsgroups/

x 20_newsgroups/talk.religion.misc/84020
x 20_newsgroups/talk.religion.misc/84021
x 20_newsgroups/talk.religion.misc/84022
x 20_newsgroups/talk.religion.misc/84023
x 20_newsgroups/talk.religion.misc/84024
x 20_newsgroups/talk.religion.misc/84025
x 20_newsgroups/talk.religion.misc/84562
x 20_newsgroups/talk.religion.misc/84563
x 20_newsgroups/talk.religion.misc/84564
x 20_newsgroups/talk.religion.misc/84565
x 20_newsgroups/talk.religion.misc/84033
x 20_newsgroups/talk.religion.misc/84042
x 20_newsgroups/talk.religion.misc/84043
x 20_newsgroups/talk.religion.misc/84047
x 20_newsgroups/talk.religion.misc/84048
x 20_newsgroups/talk.religion.misc/84052
x 20_newsgroups/talk.religion.misc/84053
x 20_newsgroups/talk.religion.misc/84169
x 20_newsgroups/talk.religion.misc/84566
x 20_newsgroups/talk.religion.misc/84054
x 20_newsgroups/talk.religion.misc/84055
x 20_newsgroups/talk.religion.misc/84170
x 20_newsgroups/talk.religion.misc/84056
x 20_newsgroups/talk.religion.misc/84057
x 20_newsgroups/

x 20_newsgroups/talk.religion.misc/84150
x 20_newsgroups/talk.religion.misc/84151
x 20_newsgroups/talk.religion.misc/84152
x 20_newsgroups/talk.religion.misc/84153
x 20_newsgroups/talk.religion.misc/84156
x 20_newsgroups/talk.religion.misc/84157
x 20_newsgroups/talk.religion.misc/84158
x 20_newsgroups/talk.religion.misc/84159
x 20_newsgroups/talk.religion.misc/84164
x 20_newsgroups/talk.religion.misc/84165
x 20_newsgroups/talk.religion.misc/84185
x 20_newsgroups/talk.religion.misc/84186
x 20_newsgroups/talk.religion.misc/84187
x 20_newsgroups/talk.religion.misc/84188
x 20_newsgroups/talk.religion.misc/84189
x 20_newsgroups/talk.religion.misc/84190
x 20_newsgroups/talk.religion.misc/84191
x 20_newsgroups/talk.religion.misc/84192
x 20_newsgroups/talk.religion.misc/84193
x 20_newsgroups/talk.religion.misc/84194
x 20_newsgroups/talk.religion.misc/84195
x 20_newsgroups/talk.religion.misc/84196
x 20_newsgroups/talk.religion.misc/84197
x 20_newsgroups/talk.religion.misc/84198
x 20_newsgroups/

We will study the full dataset, but if you want to run things a bit faster you can sample only a fraction of the dataset.  The embeddings and clusters will be different, however.

In [3]:
val corpusFraction = 1
val seed = 0L
val corpusFull: RDD[String] = sc.wholeTextFiles("20_newsgroups/*/*").map(_._2)

val corpus = if (corpusFraction == 1.0){corpusFull} else{
               corpusFull.sample(fraction = corpusFraction, withReplacement = false, seed = seed)}

corpusFraction = 1
seed = 0
corpusFull = MapPartitionsRDD[2] at map at <console>:46
corpus = MapPartitionsRDD[2] at map at <console>:46


lastException: Throwable = null


MapPartitionsRDD[2] at map at <console>:46

The first document of the corpus if from the `talk.politics.misc directory`, with the filename 179112 

In [4]:
corpus.take(1)

Array("Newsgroups: talk.politics.mideast
Path: cantaloupe.srv.cs.cmu.edu!crabapple.srv.cs.cmu.edu!bb3.andrew.cmu.edu!news.sei.cmu.edu!cis.ohio-state.edu!zaphod.mps.ohio-state.edu!cs.utexas.edu!uunet!brunix!doorknob!hm
From: hm@cs.brown.edu (Harry Mamaysky)
Subject: Heil Hernlem
In-Reply-To: hernlem@chess.ncsu.edu's message of Wed, 14 Apr 1993 12:58:13 GMT
Message-ID: <HM.93Apr15112701@yoda.cs.brown.edu>
Sender: news@cs.brown.edu
Organization: Dept. of Computer Science, Brown University
References: <1993Apr14.125813.21737@ncsu.edu>
Date: Thu, 15 Apr 1993 16:27:01 GMT
Lines: 24
In article <1993Apr14.125813.21737@ncsu.edu> hernlem@chess.ncsu.edu (Brad Hernlem) writes:
   Lebanese resistance forces detonated a bomb under an Israeli occupation
   patrol in Lebanese t...


In [5]:
//corpus.persist()

Name: Syntax Error.
Message: 
StackTrace: 

# Data Preparation

First we filter out the small words (<3 letters), and then we convert the corpus into a DataFrame, which is what the ML library uses.

In [5]:
val documentDF = corpus.map(_.toLowerCase.split("\\s")).
                        map(_.filter(_.length > 3).filter(_.forall(java.lang.Character.isLetter))).
                        toDF("text")

documentDF = [text: array<string>]


[text: array<string>]

Here is what the first article looks like after preparation.

In [6]:
documentDF.select("text").show(1,false)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|text                                                                                                                                                                                                                                              

## Word2Vec Training

The training of the Word2Vec model is more compute intensive than the actual clustering.  This is because the embedding uses a neural network.  For now, we will consider this as a mapping between groups of words and a vector.  A brief but helpful discussion of Word2Vec can be found at  https://spark.apache.org/docs/2.3.0/mllib-feature-extraction.html#word2vec, and many online resources are available.

Training should take around 10 minutes.

Defaults
- min count = 5
    - The minimum number of times a word must appear in the corpus to be counted as vocabulary
- vector size = 100
    - The dimension of the output vector. 
- Window Size is 5
    - The number of words in a window used for input training.  

In [7]:
val t0TermCount = System.nanoTime
val word2Vec = new Word2Vec().
    setInputCol("text").
    setOutputCol("result").
    setVectorSize(100).
    setMinCount(10)
val model = word2Vec.fit(documentDF)
val dtTermCount = round(System.nanoTime - t0TermCount)
println("time to generate word2vec: " + dtTermCount + "seconds")

time to generate word2vec: 485.951seconds


t0TermCount = 102542290877509
word2Vec = w2v_50799811b4ff
model = w2v_50799811b4ff
dtTermCount = 485.951


485.951

The application of the Word2Vec algorithm has the following syntax in the ML Pipeline API

In [8]:
val result = model.transform(documentDF)

result = [text: array<string>, result: vector]


[text: array<string>, result: vector]

In [9]:
result.printSchema

root
 |-- text: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- result: vector (nullable = true)



The left column is the processed article and the right column is the vector representation of the collection of words.  While the input can have many words (using a "one hot encoding" style of input), the right column always has the same number of features (100 in this case).

In [10]:
result.show()

+--------------------+--------------------+
|                text|              result|
+--------------------+--------------------+
|[heil, hernlem, m...|[-0.0376199796761...|
|[expansion, cyber...|[0.02261533695732...|
|[million, readers...|[0.01369020007739...|
|[water, thinking,...|[0.01518867839393...|
|[stage, being, co...|[0.06908274426142...|
|[zionism, racism,...|[0.02542680181795...|
|[kaya, seventh, c...|[0.01524961188708...|
|[armenians, form,...|[0.02590924940932...|
|[israelis, murder...|[0.03341428354602...|
|[muslim, women, c...|[0.00145620694723...|
|[public, service,...|[0.07969767091174...|
|[expansion, berke...|[0.04429754618242...|
|[calif, state, in...|[0.01111384224730...|
|[accounts, human,...|[0.01048161660526...|
|[saudi, clergy, c...|[0.04459902183885...|
|[islam, borders, ...|[0.04552383236099...|
|[department, redu...|[0.02646110018914...|
|[armenia, says, c...|[-0.0040151345181...|
|[wilfrid, laurier...|[-0.0184614246457...|
|[boucher, murphy,...|[-0.004481

Here we show the first article only:

In [11]:
result.show(1,false)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Now that we have a representation of our words as feature vectors, we can cluster them into "topics", or clusters, and begin to explore the internal structure that the Word2Vec and KMeans algorithms reveal.

In [13]:
//val K = math.floor(math.sqrt(documentDF.count/2)).toInt
val K = 10

K = 10


10

The KMeans algorithm is called in this notebook using the ML Pipeline API, which leverages the DataFrames API and has better performance in general.  

In [14]:
import org.apache.spark.ml.clustering.KMeans

In [15]:
val t0ClusterCount = System.nanoTime
         
val resultW2V = result.withColumnRenamed("result","features")
val kmeans = new KMeans().setK(K).setSeed(1L)
val modelK = kmeans.fit(resultW2V)
val labelsDF = modelK.transform(resultW2V).withColumnRenamed("prediction","labels")


val dtClusterCount = round(System.nanoTime - t0ClusterCount)
println("number of Clusters (k) used: " + K)
println("time to generate cluster: " + dtTermCount + "seconds")

number of Clusters (k) used: 10
time to generate cluster: 485.951seconds


t0ClusterCount = 106921276251548
resultW2V = [text: array<string>, features: vector]
kmeans = kmeans_bdc7874017b2
modelK = kmeans_bdc7874017b2
labelsDF = [text: array<string>, features: vector ... 1 more field]
dtClusterCount = 267.94


267.94

In [16]:
resultW2V.show()

+--------------------+--------------------+
|                text|            features|
+--------------------+--------------------+
|[clinton, press, ...|[-0.0502035705813...|
|[engineer, cowboy...|[-0.0190240844676...|
|[wiretapping, ini...|[-0.0232437958475...|
|[tieing, abortion...|[-0.0046298521373...|
|[expansion, mcgil...|[0.04100256933167...|
|[welcome, police,...|[0.00462032424805...|
|[role, national, ...|[0.02762026362249...|
|[concentrate, chi...|[-0.0356977651354...|
|[welcome, police,...|[-0.0049290160980...|
|[holocaust, memor...|[-0.0448481236408...|
|[real, source, tr...|[-0.0652494278806...|
|[study, percentag...|[0.00148526637805...|
|[abortion, privat...|[-0.0511604974162...|
|[earth, also, cel...|[-0.0254515306928...|
|[limiting, govt, ...|[-0.0213143074118...|
|[study, percentag...|[0.00635822871866...|
|[good, neighbor, ...|[-0.0191137050073...|
|[proof, viability...|[-0.0847958023401...|
|[proof, viability...|[-8.9741921466257...|
|[race, violence, ...|[-0.005263

We can now display each article, along with its feature vector, and the cluster number that it was assigned to.

In [17]:
labelsDF.show()

+--------------------+--------------------+------+
|                text|            features|labels|
+--------------------+--------------------+------+
|[clinton, press, ...|[-0.0502035705813...|     3|
|[engineer, cowboy...|[-0.0190240844676...|     2|
|[wiretapping, ini...|[-0.0232437958475...|     3|
|[tieing, abortion...|[-0.0046298521373...|     2|
|[expansion, mcgil...|[0.04100256933167...|     2|
|[welcome, police,...|[0.00462032424805...|     3|
|[role, national, ...|[0.02762026362249...|     2|
|[concentrate, chi...|[-0.0356977651354...|     2|
|[welcome, police,...|[-0.0049290160980...|     2|
|[holocaust, memor...|[-0.0448481236408...|     7|
|[real, source, tr...|[-0.0652494278806...|     7|
|[study, percentag...|[0.00148526637805...|     2|
|[abortion, privat...|[-0.0511604974162...|     3|
|[earth, also, cel...|[-0.0254515306928...|     7|
|[limiting, govt, ...|[-0.0213143074118...|     3|
|[study, percentag...|[0.00635822871866...|     2|
|[good, neighbor, ...|[-0.01911

Let's take a look at the first cluster:

In [18]:
labelsDF.filter("labels = 0").show()

+--------------------+--------------------+------+
|                text|            features|labels|
+--------------------+--------------------+------+
|[hitachi, oscilli...|[-0.0834846808010...|     0|
|[atari, processor...|[-0.0448761401124...|     0|
|[voice, synth, ch...|[-0.0995623438308...|     0|
|[oscilloscope, sa...|[-0.1196322398347...|     0|
|[circuit, inquiri...|[-0.0765004193420...|     0|
|[dram, controller...|[-0.0587306980641...|     0|
|[need, help, find...|[-0.0456011835331...|     0|
|[oscilliscopes, s...|[-0.1171103589321...|     0|
|[casio, serial, l...|[-0.0594486877150...|     0|
|[cross, compilers...|[-0.0312366772975...|     0|
|[video, free, uni...|[-0.0092522239512...|     0|
|[circuit, boards,...|[-0.0191166970813...|     0|
|[drive, array, wo...|[-0.0032491331498...|     0|
|[remote, extender...|[-0.0429821987527...|     0|
|[amplifier, into,...|[-0.0051534950528...|     0|
|[power, supply, s...|[0.00759276533217...|     0|
|[telephone, circu...|[-0.03784

That view only shows us the first few words, so let's use the expanded display version:

In [19]:
labelsDF.select("text").filter("labels = 0").show(false)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

It sure looks like this particular cluster maps very closely to the contents of the `sci.electronics` directory.  Perhaps this cluster could be a similar topic?  It seems like a reasonable place to start when trying to work through the text. 

Let's take a look at how many articles are in each topic:

In [21]:
labelsDF.groupBy("labels").count().orderBy("labels").show()

+------+-----+
|labels|count|
+------+-----+
|     0| 1278|
|     1| 1222|
|     2| 3568|
|     3| 2330|
|     4|  481|
|     5| 1521|
|     6| 3904|
|     7| 2180|
|     8| 2888|
|     9|  625|
+------+-----+



One very nice result here is that the clusters seem to be somewhat uniformly distributed...each cluster has a significant population, and no one cluster seems to have aggregated too many data points.  It's only the beginning here, but we might be able to identify topics with this approach!

In [22]:
labelsDF.count()

19997

### Explore Work Similarities

In [16]:
val wordVectorsDF = model.getVectors
val vocabSize = wordVectorsDF.count()
println("Vocabulary Size: "+vocabSize)

Vocabulary Size: 14896


wordVectorsDF = [word: string, vector: vector]
vocabSize = 14896


14896

In [18]:
wordVectorsDF.printSchema()

root
 |-- word: string (nullable = true)
 |-- vector: vector (nullable = true)



In [25]:
wordVectorsDF.select("word").show()

+------------+
|        word|
+------------+
|   professed|
|    incident|
|     misfire|
|     serious|
|     acronym|
|      comply|
|      breaks|
|   forgotten|
|    precious|
|       mario|
|   controler|
|inflammatory|
|     sectors|
|   ascension|
|      teresa|
|       boggs|
|       tripe|
|    embedded|
|  respecting|
|      daigle|
+------------+
only showing top 20 rows



In [26]:
model.explainParams()

vectorSize: the dimension of codes after transforming from wo...


inputCol: input column name (current: text)
maxIter: maximum number of iterations (>= 0) (default: 1)
maxSentenceLength: Maximum length (in words) of each sentence in the input data. Any sentence longer than this threshold will be divided into chunks up to the size (> 0) (default: 1000)
minCount: the minimum number of times a token must appear to be included in the word2vec model's vocabulary (>= 0) (default: 5, current: 10)
numPartitions: number of partitions for sentences of words (> 0) (default: 1)
outputCol: output column name (default: w2v_9b6a30b49a60__output, current: result)
seed: random seed (default: -1961189076)
stepSize: Step size to be used for each iteration of optimization (> 0) (default: 0.025)
vectorSize: the dimension of codes after transforming from words (> 0) (default: 100, current: 100)
windowSize: the window size (context words from [-window, window]) (> 0) (default: 5)

Word2Vec is an interesting model in that it can take an arbitrary number of words and return a vector.  The input is one very large vector of $1$s and $0$s, where each row of the vector corresponts to a word in the learned vocabulary.  Thus, an article can have many words that go into the input vector, or it can have only a single word.  The single word inputs are interesting because they can be used to find synonyms.  Let's find some synonyms to some words from our learned model:

In [27]:
val word = "drive"
val topN = 5
val synonymsDF = model.findSynonyms(word, topN)
synonymsDF.show()

+----------+------------------+
|      word|        similarity|
+----------+------------------+
|    floppy|0.7424999475479126|
|    drives|0.7419173121452332|
|controller|0.7130770087242126|
|      scsi|0.7129872441291809|
|      esdi|0.6894485950469971|
+----------+------------------+



word = drive
topN = 5
synonymsDF = [word: string, similarity: double]


[word: string, similarity: double]

In [28]:
val word = "network"
val topN = 5
val synonymsDF = model.findSynonyms(word, topN)
synonymsDF.show()

+---------+------------------+
|     word|        similarity|
+---------+------------------+
|     arpa|0.6670048832893372|
|workplace|  0.61444491147995|
|  pathway|0.5837959051132202|
|   novell|0.5518525838851929|
|   encore|0.5424853563308716|
+---------+------------------+



word = network
topN = 5
synonymsDF = [word: string, similarity: double]


[word: string, similarity: double]

Finally, let's save our work and look at another topic modeling approach...the Latent Dirichlet Allocation!

In [29]:
model.write.overwrite.save("model/word2vec")